In [ ]:
# This is necessary to recognize the modules
import os
import sys
import warnings
import pickle

warnings.filterwarnings("ignore")
root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)


In [ ]:
from core.backtesting import BacktestingEngine
import os
import datetime

EXPERIMENT_NAME = "la_maquinita"
TRADE_COST = 0.0006

start_time = datetime.datetime(2024, 8, 26).timestamp()
end_time = datetime.datetime(2024, 8, 31).timestamp()

controllers_conf_dir_path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "configs")
config_files = os.listdir(controllers_conf_dir_path)

backtesting_engine = BacktestingEngine(root_path=root_path, load_cached_data=False)
bt_results = []

for i, config_file in enumerate(config_files):
    print(f"Experiment {i}/{len(config_files)}: {config_file}")
    try:
        backtesting_results = await backtesting_engine.backtest_controller_from_yml(
            config_file=config_file,
            controllers_conf_dir_path=controllers_conf_dir_path,
            trade_cost=TRADE_COST,
            start=int(start_time), end=int(end_time), backtesting_resolution="1m")
        print(backtesting_results.get_results_summary())
        bt_results.append(backtesting_results)
    except Exception as e:
        print(f"Error with {config_file}: {e}")


path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "backtesting_results_2.pickle")
with open(f"bt_results_{EXPERIMENT_NAME}", "wb") as f:
    pickle.dump(bt_results, f)